I'm as a part of a crisis response team want to identify how hospitals have been responding to crash collisions in New York City.


In [2]:
import math
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon
import folium
from folium import Choropleth, Marker
from folium.plugins import HeatMap, MarkerCluster 


At first I load a GeoDataFrame <b>collisions</b> tracking major motor vehicle collisions in 2013-2018.

In [3]:
collisions = gpd.read_file("./NYPD_Motor_Vehicle_Collisions/NYPD_Motor_Vehicle_Collisions.shp")
collisions.head()

,DATE,TIME,BOROUGH,ZIP CODE,LATITUDE,LONGITUDE,LOCATION,ON STREET,CROSS STRE,OFF STREET,...,CONTRIBU_2,CONTRIBU_3,CONTRIBU_4,UNIQUE KEY,VEHICLE TY,VEHICLE _1,VEHICLE _2,VEHICLE _3,VEHICLE _4,geometry
0,07/30/2019,0:00,BRONX,10464,40.841100,-73.784960,"(40.8411, -73.78496)",None,None,121 PILOT STREET,...,Unspecified,None,None,4180045,Sedan,Station Wagon/Sport Utility Vehicle,Station Wagon/Sport Utility Vehicle,None,None,POINT (1043750.211 245785.815)
1,07/30/2019,0:10,QUEENS,11423,40.710827,-73.770660,"(40.710827, -73.77066)",JAMAICA AVENUE,188 STREET,None,...,None,None,None,4180007,Sedan,Sedan,None,None,None,POINT (1047831.185 198333.171)
2,07/30/2019,0:25,None,None,40.880318,-73.841286,"(40.880318, -73.841286)",BOSTON ROAD,None,None,...,None,None,None,4179575,Sedan,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (1028139.293 260041.178)
3,07/30/2019,0:35,MANHATTAN,10036,40.756744,-73.984590,"(40.756744, -73.98459)",None,None,155 WEST 44 STREET,...,None,None,None,4179544,Box Truck,Station Wagon/Sport Utility Vehicle,None,None,None,POINT (988519.261 214979.32)
4,07/30/2019,10:00,BROOKLYN,11223,40.600090,-73.965910,"(40.60009, -73.96591)",AVENUE T,OCEAN PARKWAY,None,...,None,None,None,4180660,Station Wagon/Sport Utility Vehicle,Bike,None,None,None,POINT (993716.669 157907.212)


I use the "LATITUDE" and "LONGITUDE" columns to create an interactive map to visualize the collision data. The most effective type of map would be a Heat Map

In [15]:
map = folium.Map(location=[40.7, -74], zoom_start=11) 

HeatMap(data=collisions[['LATITUDE', 'LONGITUDE']], radius=9).add_to(map)

map.save('index.html')

After that I load the <b>hospital</b> data.

In [5]:
hospitals = gpd.read_file("./nyu_2451_34494/nyu_2451_34494.shp")
hospitals.head()

,id,name,address,zip,factype,facname,capacity,capname,bcode,xcoord,ycoord,latitude,longitude,geometry
0,317000001H1178,BRONX-LEBANON HOSPITAL CENTER - CONCOURSE DIVI...,1650 Grand Concourse,10457,3102,Hospital,415,Beds,36005,1008872.0,246596.0,40.843490,-73.911010,POINT (1008872 246596)
1,317000001H1164,BRONX-LEBANON HOSPITAL CENTER - FULTON DIVISION,1276 Fulton Ave,10456,3102,Hospital,164,Beds,36005,1011044.0,242204.0,40.831429,-73.903178,POINT (1011044 242204)
2,317000011H1175,CALVARY HOSPITAL INC,1740-70 Eastchester Rd,10461,3102,Hospital,225,Beds,36005,1027505.0,248287.0,40.848060,-73.843656,POINT (1027505 248287)
3,317000002H1165,JACOBI MEDICAL CENTER,1400 Pelham Pkwy,10461,3102,Hospital,457,Beds,36005,1027042.0,251065.0,40.855687,-73.845311,POINT (1027042 251065)
4,317000008H1172,LINCOLN MEDICAL & MENTAL HEALTH CENTER,234 E 149 St,10451,3102,Hospital,362,Beds,36005,1005154.0,236853.0,40.816758,-73.924478,POINT (1005154 236853)


Foe now I use the "latitude" and "longitude" columns to visualize the hospital locations.

In [16]:
map = folium.Map(location=[40.7, -74], zoom_start=11) 

for idx,row in hospitals.iterrows():
    Marker([row['latitude'], row['longitude']]).add_to(map)

map.save('index1.html')

I want to create a DataFrame <b>outside_range</b> containing all rows from collisions with crashes that occurred more than 10 kilometers from the closest hospital. So firstly I create a series of <b>buffer polygons</b> around every hospital with the radius of 10 km. Then I need to collapse them all into MultiPolygon <b>un</b>.

!Both hospitals and collisions have EPSG 2263 as the coordinate reference system, and EPSG 2263 has units of meters.

In [7]:
buf = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10*1000)
un = buf.geometry.unary_union
outside_range = collisions.loc[~collisions["geometry"].apply(lambda x: un.contains(x))]


C:\Users\polin\AppData\Local\Temp\ipykernel_17828\1438944819.py:2: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  un = buf.geometry.unary_union


The next code cell calculates the <b>percentage</b> of collisions that occurred more than 10 kilometers away from the closest hospital

In [8]:
percentage = round(100*len(outside_range)/len(collisions), 2)
print("Percentage of collisions more than 10 km away from the closest hospital: {}%".format(percentage))

Percentage of collisions more than 10 km away from the closest hospital: 15.12%


We see, thet this percetnage is rather large. 

When collisions occur in distant locations, it becomes even more vital that injured persons are transported to the nearest available hospital. With this in mind, I decide to create a recommender that:

- takes the location of the crash (in EPSG 2263) as input,

- finds the closest hospital (where distance calculations are done in EPSG 2263), and

- returns the name of the closest hospital.

In [11]:
def best_hospital(collision_location):
    """Finds the distance between concrete collision and all hospitals. 
    Then finds the minimal distance - closest hospital - and takes it as index.
    Takes only the 'name' column of hospital found"""
    dist = hospitals.geometry.distance(collision_location)
    name = hospitals.iloc[dist.idxmin()]['name']
    return name


Considering only collisions in the <b>outside_range</b> DataFrame, I would like to know, which hospital is most recommended.


In [12]:
highest_demand = outside_range.geometry.apply(best_hospital).value_counts().idxmax()

highest_demand

'JAMAICA HOSPITAL MEDICAL CENTER'

I visualize <b>hospital locations</b> and its <b>buffer polygons</b>, in addition to <b>collisions</b> that occurred more than 10 kilometers away from the closest hospital(as a Heat Map). There is a possibility to click anywhere on the map to see a pop-up with the corresponding location in latitude and longitude.

In [17]:
map= folium.Map(location=[40.7, -74], zoom_start=11) 

buf = gpd.GeoDataFrame(geometry=hospitals.geometry).buffer(10000)
folium.GeoJson(buf.geometry.to_crs(epsg=4326)).add_to(map)
HeatMap(data=outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(map)
folium.LatLngPopup().add_to(map)

map.save('index2.html')

The city of New York reaches out to me for help with deciding locations for two brand new hospitals. They specifically want my help with identifying locations to bring the calculated percentage to <b>less than ten percent</b>. Using the map (and without worrying about zoning laws or what potential buildings would have to be removed in order to build the hospitals), I can identify two locations that would help the city accomplish this goal. I put the proposed latitude and longitude for hospital 1 in lat_1 and long_1, respectively. (Likewise for hospital 2.)

In [18]:
# proposed location of hospital 1
lat_1 = 40.6744
long_1 = -73.8666

# proposed location of hospital 2
lat_2 = 40.6931
long_2 = -73.7464

new_df = pd.DataFrame(
    {'Latitude': [lat_1, lat_2],
     'Longitude': [long_1, long_2]})
new_gdf = gpd.GeoDataFrame(new_df, geometry=gpd.points_from_xy(new_df.Longitude, new_df.Latitude))
new_gdf.crs = {'init' :'epsg:4326'}
new_gdf = new_gdf.to_crs(epsg=2263)
# get new percentage
new_coverage = gpd.GeoDataFrame(geometry=new_gdf.geometry).buffer(10000)
new_my_union = new_coverage.geometry.unary_union
new_outside_range = outside_range.loc[~outside_range["geometry"].apply(lambda x: new_my_union.contains(x))]
new_percentage = round(100*len(new_outside_range)/len(collisions), 2)
print("(NEW) Percentage of collisions more than 10 km away from the closest hospital: {}%".format(new_percentage))
# make the map
map = folium.Map(location=[40.7, -74], zoom_start=11) 
folium.GeoJson(buf.geometry.to_crs(epsg=4326)).add_to(map)
folium.GeoJson(new_coverage.geometry.to_crs(epsg=4326)).add_to(map)
for idx, row in new_gdf.iterrows():
    Marker([row['Latitude'], row['Longitude']]).add_to(map)
HeatMap(data=new_outside_range[['LATITUDE', 'LONGITUDE']], radius=9).add_to(map)
folium.LatLngPopup().add_to(map)
map.save('index3.html')


c:\Users\polin\AppData\Local\Programs\Python\Python311\Lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\polin\AppData\Local\Temp\ipykernel_17828\3851346523.py:17: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  new_my_union = new_coverage.geometry.unary_union


(NEW) Percentage of collisions more than 10 km away from the closest hospital: 9.42%
